## Import Libraries

In [1]:
import os
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from cv2 import resize as cv2_resize
from librosa import load as lib_load
from librosa import feature as lib_feature

# Import sci-kit models
from sklearn.preprocessing import OneHotEncoder 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix

## Load Data

In [2]:
# Launch Jupyter Notebook from the project folder
root_path = os.getcwd()
# Data is stored in the folder named "Data"
path = os.path.join(root_path, 'Data')

In [3]:
def load_data(path, sample_rate):
    # Create data lists
    samples = []
    labels = []
    classes = os.listdir(path)
    print('Loading data...')

    for file in os.listdir(path):
        class_path = os.path.join(path, file)
        for filename in os.listdir(class_path):
            # Load data
            y, s = lib_load(os.path.join(class_path,filename), sr=sample_rate)
            # Append data and label
            labels.append([file])
            samples.append(y)
            #print('Loaded {}'.format(filename))
            
    return samples, classes, labels

In [4]:
# Define sample rate
sample_rate = 44_100
# Load data
samples, classes, labels = load_data(path, sample_rate)

# Print data information
print('Loaded data from the directory {} \n'.format(path))
print('Loaded the classes: {} \n'.format(classes))
print('Loaded {} samples with {} labels \n'.format(len(samples), len(labels)))

# Check lengths of samples and their labels
for idx, sample in enumerate(samples):
  print('Sample {:2}: \t {:.1f}s \t {}'.format(idx, len(sample)/sample_rate, labels[idx]))

Loading data...
Loaded data from the directory C:\Users\14w\Documents\CS522_final\Data 

Loaded the classes: ['Dendropsophus bifurcus', 'Engystomops petersi', 'Pristimantis conspicillatus'] 

Loaded 31 samples with 31 labels 

Sample  0: 	 105.8s 	 ['Dendropsophus bifurcus']
Sample  1: 	 593.1s 	 ['Dendropsophus bifurcus']
Sample  2: 	 45.9s 	 ['Dendropsophus bifurcus']
Sample  3: 	 29.4s 	 ['Dendropsophus bifurcus']
Sample  4: 	 105.8s 	 ['Dendropsophus bifurcus']
Sample  5: 	 240.3s 	 ['Dendropsophus bifurcus']
Sample  6: 	 313.6s 	 ['Dendropsophus bifurcus']
Sample  7: 	 409.2s 	 ['Dendropsophus bifurcus']
Sample  8: 	 208.7s 	 ['Dendropsophus bifurcus']
Sample  9: 	 317.8s 	 ['Dendropsophus bifurcus']
Sample 10: 	 512.6s 	 ['Engystomops petersi']
Sample 11: 	 419.5s 	 ['Engystomops petersi']
Sample 12: 	 368.0s 	 ['Engystomops petersi']
Sample 13: 	 320.0s 	 ['Engystomops petersi']
Sample 14: 	 641.0s 	 ['Engystomops petersi']
Sample 15: 	 1018.6s 	 ['Engystomops petersi']
Sample 1

## Encode labels

In [5]:
# Reshape classes list
unique_classes = [[c] for c in classes]

# Create encoder, fit, and check encoding
enc = OneHotEncoder()
enc.fit(unique_classes)
enc_labels = enc.transform(labels).toarray()
for idx, item in enumerate(enc_labels):
    print('Encoding is {}, label is {}'.format(enc_labels[idx], labels[idx]))

Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [1. 0. 0.], label is ['Dendropsophus bifurcus']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
Encoding is [0. 1. 0.], label is ['Engystomops petersi']
E

## Get Spectrograms

In [6]:
def get_spectrograms(time_data, FFT_SIZE=1024, fs=44100):
    ## Compute Fast Fourier Transform of data
    # Create FFT list
    pxx = []
    
    print("Calculating FFTs for data ...")
    for sample in time_data:
        _, _, pxx_idx = signal.spectrogram(sample, nperseg=FFT_SIZE, fs=fs, noverlap=FFT_SIZE/2)
        
        pxx.append(pxx_idx)

    return pxx.copy()

In [9]:
# Specify FFT size
fft_size = 1024
# Load data
specs = get_spectrograms(samples, fft_size, sample_rate)

# Check spectrogram data shapes
print('Spectrograms: \t {}'.format(len(specs)))
print('Frequency bins: {} \n'.format(len(specs[0])))
for idx, spec in enumerate(specs):
    print('Spectrogram {} time bins: \t {}'.format(idx, len(spec[0])))

Calculating FFTs for data ...
Spectrograms: 	 31
Frequency bins: 513 

Spectrogram 0 time bins: 	 9111
Spectrogram 1 time bins: 	 51086
Spectrogram 2 time bins: 	 3954
Spectrogram 3 time bins: 	 2529
Spectrogram 4 time bins: 	 9111
Spectrogram 5 time bins: 	 20699
Spectrogram 6 time bins: 	 27013
Spectrogram 7 time bins: 	 35243
Spectrogram 8 time bins: 	 17975
Spectrogram 9 time bins: 	 27375
Spectrogram 10 time bins: 	 44146
Spectrogram 11 time bins: 	 36130
Spectrogram 12 time bins: 	 31698
Spectrogram 13 time bins: 	 27558
Spectrogram 14 time bins: 	 55212
Spectrogram 15 time bins: 	 87730
Spectrogram 16 time bins: 	 43410
Spectrogram 17 time bins: 	 5782
Spectrogram 18 time bins: 	 6871
Spectrogram 19 time bins: 	 11389
Spectrogram 20 time bins: 	 30733
Spectrogram 21 time bins: 	 19332
Spectrogram 22 time bins: 	 22743
Spectrogram 23 time bins: 	 19580
Spectrogram 24 time bins: 	 17515
Spectrogram 25 time bins: 	 26487
Spectrogram 26 time bins: 	 10321
Spectrogram 27 time bins: 	

## Binning Data

In [10]:
def get_bins(spec_data, num_freq_bins=5, num_time_bins=5):
    binned_data = []
        
    # Get individual samples
    for s in spec_data:
        #Open CV's resize takes (columns,rows) as the input for desired size
        resized_pxx=cv2_resize(s[:,:],(num_time_bins,num_freq_bins))
        binned_data.append(resized_pxx.flatten())
            
    return binned_data

In [11]:
# Define bins
freq_bins = 5
time_bins = 5
# Get binned data
binned_specs = get_bins(specs, freq_bins, time_bins)

# Print bin shapes
print('Binned spectrograms: \t {}'.format(len(binned_specs)))
print('Number of bins: \t {}'.format(len(binned_specs[0])))

Binned spectrograms: 	 31
Number of bins: 	 25


## Get Freq-Windows

In [12]:
def get_freq_windows(time_data, len_window=1, overlap=0, sample_rate=10):
    len_data = time_data.shape[1]
#     print(len_data)
    
    windows = []
    
    # How far the window should shift
    if overlap == 0:
        step = len_window
        
        start = 0
        stop = step
#         print(len_window)
        for n in range(int(len_data/len_window)):
            windows.append(time_data[:, start:stop])
            
#             print(start)
#             print(stop)
#             print()
            start = start + step
            stop = stop + step
            
            if start > len_data or stop > len_data:
                break
    else:
        step = int(len_window*overlap)
        
        start = 0
        stop = len_window
        
#         print(len_window)
        for n in range(int(len_data/step)):
            windows.append(time_data[:, start:stop])
            
#             print(start)
#             print(stop)
#             print()
            start = start + step
            stop = stop + step
            
            if start > len_data or stop > len_data:
                break
        
    return windows

In [13]:
windowed_freq = []
for fft in specs:
    window_freq_data = get_windows(fft, len_window=2, overlap=.5)
    #shape = np.array(window_data).shape
    #window_data = np.array(window_data).reshape(shape[0], shape[1]*shape[2]) 
    print(np.array(window_data).shape)
    windowed_freq.append(window_data)  

NameError: name 'get_windows' is not defined

## Get Time-Windows

In [ ]:
def get_time_windows(time_data, len_window_sec=1, overlap=0):
  len_data = time_data.shape[0]
  # print(len_data)

  len_window = len_window_sec*44100
    
  windows = []
  # How far the window should shift
  if overlap == 0:
      step = len_window
      
      start = 0
      stop = step
      # print(len_window)

      for n in range(int(len_data/len_window)):
          windows.append(time_data[start:stop])
          
          # print(start)
          # print(stop)
          # print()
          start = start + step
          stop = stop + step
          
          if start > len_data or stop > len_data:
              break
  else:
      step = int(len_window*overlap)
      
      start = 0
      stop = len_window
      
  #         print(len_window)
      for n in range(int(len_data/step)):
          windows.append(time_data[start:stop])
          
  #             print(start)
  #             print(stop)
  #             print()
          start = start + step
          stop = stop + step
          
          if start > len_data or stop > len_data:
              break
        
  return windows

time_windows = []
num_labels = []
for sample in samples:
  new_windows = get_time_windows(samples[0], len_window_sec=5)
  num_labels.append(len(new_windows))
  print(np.array(new_windows).shape)
  time_windows = time_windows + new_windows

In [ ]:
new_labels = []
for label in labels:
  new_labels = new_labels + [label for num in num_labels]

enc_labels = enc.transform(new_labels).toarray()
print('Encoding is:')
print(enc_labels)

In [ ]:
train, test = cv.split(np.array(X), np.array(y.argmax(axis=1)))

## Feature Extraction

In [ ]:
specs_reshape = []
for i in range(len(specs)):
  shape = np.array(specs[i]).shape
  X = np.array(specs[i]).reshape(shape[0]*shape[1])
  specs_reshape.append(X)
  

In [ ]:
window_numbers = 20 ## need to be determined
domain_fv = []

for i in range(len(specs_reshape)):
  #specs reshape: [frequency][time]
  print(specs_reshape[0])
  
    # peak frequency
      for 

    # zero crossing rate
    # threshold crossing rate bandwidth

    # MFCC
    mfccs = lib_feature.mfcc(y=specs_reshape[i], sr=44100, n_mfcc=20, win_length = int(np.ceil(1024/window_numbers)), hop_length = int(np.ceil(1024/(2*window_numbers))))
    mfccs_mean = np.mean(mfccs.T, axis=0)
    #mfccs_mean=mfccs_mean.reshape(513,20)

    # Spectral Centroid
#     sc = lib_feature.spectral_centroid(y=spectrograms[i], sr=44100, win_length = int(np.ceil(1024/5)), hop_length = int(np.ceil(1024/10)))
#     reshape_sc=[]
#     for x in sc:
#         for j in x:
#             reshape_sc.append(j)

    #all_features=list(zip(mfccs_mean,reshape_sc))
    domain_fv.append(mfccs_mean)
    

## Train Model

In [ ]:
# Specify which data to use, these are the only parameters that should change, the rest should remain the same.
X = time_windows
y = enc_labels
depth = 5


# # Create our imputer to replace missing values with the mean 
# imp = SimpleImputer(missing_values=np.nan, strategy='mean')
# imp = imp.fit(X)
    
# # Impute our data
# X_imp = imp.transform(X)

# Define model
clf = RandomForestClassifier(max_depth=depth, random_state=0)

# Define number of folds
cv = StratifiedKFold(n_splits=10, shuffle=False)

# Split data, train and test model on 10 folds
split = 1
scores = []
confuse_mat = []
for train_index, test_index in cv.split(np.array(X), np.array(y.argmax(axis=1))):
    print(f"Split {split}/10...")
    x_train, y_train = X[train_index], enc_labels[train_index]
    x_test, y_test = X[test_index], enc_labels[test_index]
    
    
    # Fit model and evaluate it
    clf.fit(x_train, y_train)
    scores.append(clf.score(x_test, y_test))
    
    # Construct confusion matrix
    y_predict = clf.predict(x_test)
    confuse_mat.append(confusion_matrix(y_test.argmax(axis=1), y_predict.argmax(axis=1)))
    
    # Iterate
    split = split + 1
    
print("Mean accuracy:" + str(np.mean(scores)))

In [ ]:
# TODO: should probably do kfold but doing this just for testing purposes
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(binned_data, enc_labels, test_size=.3, stratify=enc_labels)

In [ ]:
clf = RandomForestClassifier()

clf.fit(x_train, y_train)

print(clf.score(x_test, y_test))

## Save Model

In [ ]:
import pickle

pkl_filename = "pickle_model.pkl"

# Note, for now this will only store the file in volitle memory in the notebook
# To save permanently, most add file path to drive
with open(pkl_filename, 'wb') as file:
  pickle.dump(clf, file)

## Load Model

In [ ]:
with open(pkl_filename, 'rb') as file:
  pickle_model = pickle.load(file)

In [ ]:
# Test saved model
print(pickle_model.score(x_test, y_test))